In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
def train_step(u_batch, i_batch, uid, iid, y_batch,batch_num):
    """
    A single training step
    """
    feed_dict = {                         #placeholder haro meghdardehi sabet mikone
        deep.input_u: u_batch,
        deep.input_i: i_batch,
        deep.input_uid: uid,
        deep.input_iid: iid,
        deep.input_y: y_batch,
    }
    _, step,summaries, loss, RMSE, MAE, fm,prediction,y = sess.run(
        [train_op, global_step,train_summary_op, deep.loss, deep.RMSE, deep.MAE,deep.score,deep.predictions,deep.input_y]
        , feed_dict)
     
    # time_str = datetime.datetime.now().strftime("%X")
    # print("{}: step {}, loss {:g}, rmse {:g},mae {:g}".format(time_str, batch_num, loss, RMSE, MAE))
    train_summary_writer.add_summary(summaries, step)
    return RMSE, MAE,fm,prediction,y
  
def dev_step(u_batch, i_batch, uid, iid, y_batch,writer=None):
    """
    Evaluates model on a dev set

    """
    feed_dict = {
        deep.input_u: u_batch,
        deep.input_i: i_batch,
        deep.input_y: y_batch,
        deep.input_uid: uid,
        deep.input_iid: iid,
    }
    step,summaries, loss, RMSE, mae,prediction,y = sess.run(
        [global_step,dev_summary_op, deep.loss, deep.RMSE, deep.MAE,deep.predictions,deep.input_y], feed_dict)
    # time_str = datetime.datetime.now().strftime("%X")
#     print("{}: step{}, loss {:g}, rmse {:g},mae {:g}".format(time_str, step, loss, accuracy, mae))
    if writer:
        writer.add_summary(summaries, step)
    return [loss, RMSE, mae,prediction,y]  

In [4]:
import numpy as np
import tensorflow as tf
import pickle
import datetime
import sys
import time
import os


word2vec="/content/drive/My Drive/dataset/glove.6B.100d.txt"
valid_data="/content/drive/My Drive/dataset/music/project/music.test"
para_data= "/content/drive/My Drive/dataset/music/project/music.para"
train_data= "/content/drive/My Drive/dataset/music/project/music.train"
# ==================================================

# Model Hyperparameters
embedding_dim= 100
l2_reg_lambda= 0.001

# Misc Parameters
allow_soft_placement= True
log_device_placement= False

print("Loading data...")

# with open (para_data, 'rb') as pkl_file:
#   pkl_file.close()
pkl_file = open(para_data, 'rb')

  
para = pickle.load(pkl_file)
user_num = para['user_num']
item_num = para['item_num']
review_num_u = para['review_num_u']
review_num_i = para['review_num_i']
review_len_u = para['review_len_u']
review_len_i = para['review_len_i']
vocabulary_user = para['user_vocab']
vocabulary_item = para['item_vocab']
train_length = para['train_length']
test_length = para['test_length']
u_text = para['u_text']
i_text = para['i_text']

np.random.seed(2017)
random_seed = 2017
print (user_num)
print (item_num)
print (review_num_u)
print (review_len_u)
print (review_num_i)
print (review_len_i)

pkl_file = open(train_data, 'rb')

train_data = pickle.load(pkl_file)

train_data = np.array(train_data)
pkl_file.close()

pkl_file = open(valid_data, 'rb')

test_data = pickle.load(pkl_file)
test_data = np.array(test_data)
pkl_file.close()

Loading data...
4992
3328
9
294
16
295


In [5]:
u=0
v=0
initW1 = np.random.uniform(-1.0, 1.0, (len(vocabulary_user), embedding_dim))

with open(word2vec,encoding="utf-8") as f:
    for line in f:
        p=line.split()
        word=p[0]
        if word in vocabulary_user:
            u = u + 1
            idx = vocabulary_user[word]
            initW1[idx] = np.asarray(p[1:], dtype="float32")
initW2 = np.random.uniform(-1.0, 1.0, (len(vocabulary_item),embedding_dim))

with open(word2vec,encoding="utf-8") as f:
    for line in f:
        p=line.split()
        word=p[0]
        if word in vocabulary_item:
            v=v+1
            idx = vocabulary_item[word]
            initW2[idx] = np.asarray(p[1:], dtype="float32")
                        
print ("in vocab_u ", u)
print ("in vocab_i ", v)

in vocab_u  9545
in vocab_i  10189


In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow as tff
class NARRE(object):
    def __init__(
            self, review_num_u, review_num_i, review_len_u, review_len_i, user_num, item_num, num_classes,
            user_vocab_size, item_vocab_size, n_latent, embedding_id, attention_size,
            embedding_size, l2_reg_lambda=0.1):
        self.input_u = tf.placeholder(tf.int32, [None, review_num_u, review_len_u], name="input_u")
        self.input_i = tf.placeholder(tf.int32, [None, review_num_i, review_len_i], name="input_i")
        self.input_y = tf.placeholder(tf.float32, [None, 1], name="input_y")
        self.input_uid = tf.placeholder(tf.int32, [None, 1], name="input_uid")
        self.input_iid = tf.placeholder(tf.int32, [None, 1], name="input_iid")

        print('input user shape:',self.input_u.shape)
        l2_loss = tf.constant(0.0)
        self.dropout_keep_prob=tf.constant(0.7)
    #############################################
        with tf.name_scope("user_embedding"):
            self.W1 = tf.Variable(tf.random_uniform([user_vocab_size, embedding_size], -1.0, 1.0),name="W1",trainable=False)
            self.embedded_user = tf.nn.embedding_lookup(self.W1, self.input_u)                            #(100,16,441,50)
            print('self.embedded_users.shape',self.embedded_user.shape)

        with tf.name_scope("item_embedding"):
            self.W2 = tf.Variable(tf.random_uniform([item_vocab_size, embedding_size], -1.0, 1.0),name="W2",trainable=False)
            self.embedded_item = tf.nn.embedding_lookup(self.W2, self.input_i)
            print('self.embedded_item.shape',self.embedded_item.shape)
   #######################################################         
        with tf.name_scope("LSTM_cell"):
            rnn_size=100
            num_layers=1
        with tf.variable_scope("scope", reuse = False ): 
            
            rnn_cell1 = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_size,name="rnn_user",reuse = tf.AUTO_REUSE)
        with tf.variable_scope("scope2",reuse = False): 
            rnn_cell2 = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_size,name="rnn_item",reuse = tf.AUTO_REUSE)

            
        with tf.name_scope("LSTM_User"):
            user_jadid=tf.reshape(self.embedded_user,[-1, review_len_u, embedding_size])
            print('user jadid shape: ',user_jadid.shape)
            self.rnn_outputs_user, user_state = tf.nn.dynamic_rnn(
                cell=rnn_cell1, inputs=user_jadid , dtype=user_jadid.dtype)
            
            self.rnn_outputs_user = tf.nn.dropout(self.rnn_outputs_user, rate=1-self.dropout_keep_prob)   # khorooji CNN user _ feature vector for  every user  - O_ul   (100,16,100)
            self.h_pool_flat_u = tf.reshape(self.rnn_outputs_user, [-1, review_num_u*review_len_u, rnn_size])  

        with tf.name_scope("LSTM_item"):
            item_jadid=tf.reshape(self.embedded_item,[-1, review_len_i, embedding_size])
            print('item jadid shape: ',item_jadid.shape)
            self.rnn_outputs_item, item_state = tf.nn.dynamic_rnn(
                cell=rnn_cell2, inputs=item_jadid ,dtype=item_jadid.dtype)
            self.rnn_outputs_item = tf.nn.dropout(self.rnn_outputs_item, rate=1-self.dropout_keep_prob)   #khorooji CNN item _ feature vector for every item        with tf.name_scope("dropout_item"):    
            self.h_pool_flat_i = tf.reshape(self.rnn_outputs_item, [-1, review_num_i*review_len_i, rnn_size])  

            # stacked = tf.nn.rnn_cell.MultiRNNCell(fw_cells, state_is_tuple=True)
            # self.rnn_outputs_user, _ = tf.nn.dynamic_rnn(
            #     cell=stacked, inputs=user_jadid , dtype=self.embedded_user.dtype)
            # print('a')
            # self.rnn_outputs_item, _ = tf.nn.dynamic_rnn(
            #     cell=stacked, inputs=item_jadid ,dtype=self.embedded_item.dtype)

            print('rnn_outputs_user shape',(self.rnn_outputs_user[0].shape,self.rnn_outputs_user[1].shape))
            print('rnn_outputs_item shape',(self.rnn_outputs_item[0].shape,self.rnn_outputs_item[1].shape))
#######################################################################

        # with tf.name_scope("dropout_user"):

        print('user_dropOut',self.rnn_outputs_user.shape) 
        print('item_dropOut',self.rnn_outputs_item.shape)

        print('flat output user:',self.h_pool_flat_u.shape )  
        print('flat output item:',self.h_pool_flat_i.shape )
###################################### ##########################           
        with tf.name_scope("attention_user"): 
            Wau = tf.Variable(tf.random_uniform([rnn_size, attention_size], -0.1, 0.1), name='Wau')  #Wo
            Wpu = tf.Variable(tf.random_uniform([attention_size, 1], -0.1, 0.1), name='Wpu')  #hT
            bau = tf.Variable(tf.constant(0.1, shape=[attention_size]), name="bau")    #b1
            bbu = tf.Variable(tf.constant(0.1, shape=[1]), name="bbu")    #b2
            self.u_j=tf.matmul(tf.nn.relu(tf.matmul(self.h_pool_flat_u, Wau)+ bau),Wpu)+bbu  # None*u_len*1   a*_ul  (100,16,1)

            self.attention_score_user = tf.nn.softmax(self.u_j,1,name="attention_score_user")  # none*u_len*1    vazne atention user   a_ul  (100,16,1)
            print('attention_score_user',self.attention_score_user.shape)
           
        with tf.name_scope("attention_item"):  

            Wai = tf.Variable( tf.random_uniform([rnn_size, attention_size], -0.1, 0.1), name='Wai')
            Wpi = tf.Variable( tf.random_uniform([attention_size, 1], -0.1, 0.1), name='Wpi')
            bai = tf.Variable(tf.constant(0.1, shape=[attention_size]), name="bai")
            bbi = tf.Variable(tf.constant(0.1, shape=[1]), name="bbi")
           
            self.i_j=tf.matmul(tf.nn.relu(tf.matmul(self.h_pool_flat_i, Wai)+ bai),Wpi)+bbi

            self.attention_score_item = tf.nn.softmax(self.i_j,1,name="attention_score_item")  # none*len*1     # vazne attention item   contribution of the lth review to the feature profile of item i:
            print('attention_score_item',self.attention_score_item.shape)

###################################################################

        with tf.name_scope("rated_review_user"):
            self.u_feast = tf.reduce_sum(tf.multiply(self.attention_score_user, self.h_pool_flat_u), 1)
            self.u_feas = tf.nn.dropout(self.u_feast, rate=1-self.dropout_keep_prob,name="rated_review_user")  # feature vector for user - user representation (Xu)
            
            print('user_feature vector:',self.u_feas.shape)
        with tf.name_scope("rated_review_item"):
            self.i_feas = tf.reduce_sum(tf.multiply(self.attention_score_item, self.h_pool_flat_i), 1)
            self.i_feas = tf.nn.dropout(self.i_feas, rate=1-self.dropout_keep_prob,name="rated_review_item")  # feature vector for item - item representation (Yi)
            print('item_feature vector:',self.i_feas.shape)
 ###################################################################           
        with tf.name_scope("feature_user"):
            uidmf = tf.Variable(tf.random_uniform([51764 + 2, embedding_id], -0.1, 0.1), name="uidmf")

            self.uid = tf.nn.embedding_lookup(uidmf,self.input_uid)   # user prefrences (q_u)  
            self.uid = tf.reshape(self.uid,[-1,embedding_id])
            Wu = tf.Variable( tf.random_uniform([rnn_size, n_latent], -0.1, 0.1), name='Wu')  #100*32
            bu = tf.Variable(tf.constant(0.1, shape=[n_latent]), name="bu")
            self.u_feas = tf.matmul(self.u_feas, Wu)+self.uid + bu    #Xu+qu
            self.u_feas = tf.identity(self.u_feas, name="feature_user")

            print('Xu+qu',self.u_feas.shape)
        with tf.name_scope("feature_item"):
            iidmf = tf.Variable(tf.random_uniform([51764 + 2, embedding_id], -0.1, 0.1), name="iidmf")
            self.iid = tf.nn.embedding_lookup(iidmf,self.input_iid)   # item features   (p_i)
            self.iid = tf.reshape(self.iid,[-1,embedding_id])
            
            Wi = tf.Variable( tf.random_uniform([rnn_size, n_latent], -0.1, 0.1), name='Wi')
            bi = tf.Variable(tf.constant(0.1, shape=[n_latent]), name="bi")
            self.i_feas = tf.matmul(self.i_feas, Wi)+self.iid + bi    # Yi+pi
            print('Yi+pi',self.i_feas.shape)
            self.i_feas = tf.identity(self.i_feas, name="feature_item")
######################################################################            
        with tf.name_scope("FM"):
            self.FM = tf.multiply(self.u_feas, self.i_feas)
            self.FM = tf.nn.relu(self.FM)
            print('FM ',self.FM.shape)
            self.FM=tf.nn.dropout(self.FM,rate=1-self.dropout_keep_prob)
# ######################################################################            
            Wmul=tf.Variable(tf.random_uniform([n_latent, 1], -0.1, 0.1), name='wmul')
            self.mul=tf.matmul(self.FM,Wmul)
            self.score=tf.reduce_sum(self.mul,1,keepdims=True,name="score")
            print('score: ',self.score.shape)

#######################################################################
        with tf.name_scope("biases"):
            self.uidW = tf.Variable(tf.constant(0.1, shape=[51764 + 2]), name="uidW")
            self.iidW = tf.Variable(tf.constant(0.1, shape=[51764 + 2]), name="iidW")
            self.u_bias = tf.gather(self.uidW, self.input_uid)
            print('self.u_bias',self.u_bias.shape)
            self.i_bias = tf.gather(self.iidW, self.input_iid)
            self.Feature_bias = self.u_bias + self.i_bias
            self.biased = tf.Variable(tf.constant(0.1), name='bias')
#######################################################################            
        with tf.name_scope("prediction"):
            self.predictions = self.score + self.Feature_bias + self.biased    # rate nahayi
            self.predictions = tf.identity(self.predictions, name="predictions")
            print('predictions',self.predictions.shape)
#######################################################################            
        with tf.name_scope("loss"):
            losses = tf.nn.l2_loss(tf.subtract(self.predictions, self.input_y))
            self.loss = losses + l2_reg_lambda * l2_loss  #amalan l2_loss=0

        with tf.name_scope("accuracy"):
            self.MAE = tf.reduce_mean(tf.abs(tf.subtract(self.predictions, self.input_y)),name="MAE")
            self.RMSE =tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.predictions, self.input_y))),name='RMSE')


In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.WARN)

import warnings
warnings.filterwarnings("ignore")
from datetime import datetime 
import pytz 
  
IST = pytz.timezone('Asia/Tehran') 
datetime_ist = datetime.now(IST) 
current_time = datetime_ist.strftime('%H%M%S')

import time
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# with tf.Graph().as_default():
session_conf = tf.ConfigProto(allow_soft_placement=allow_soft_placement, log_device_placement=log_device_placement)
session_conf.gpu_options.allow_growth = True
sess = tf.Session(config=session_conf)
with sess.as_default():
    deep = NARRE( review_num_u=review_num_u, review_num_i=review_num_i,  review_len_u=review_len_u,  review_len_i=review_len_i,
        user_num=user_num, item_num=item_num, num_classes=1, user_vocab_size=len(vocabulary_user), item_vocab_size=len(vocabulary_item),
        embedding_size=embedding_dim,  embedding_id=32,l2_reg_lambda=l2_reg_lambda, attention_size=32,  n_latent=32)
    tf.set_random_seed(random_seed)
    
    global_step = tf.Variable(0, name="global_step", trainable=False)
    # optimizer = tf.train.AdagradOptimizer(learning_rate=0.01, initial_accumulator_value=1e-8).minimize(deep.loss)
    optimizer =tf.train.AdamOptimizer(0.005) # tf.train.AdamOptimizer(0.002, beta1=0.9, beta2=0.999, epsilon=1e-8).minimize(deep.loss)
    grads_and_vars = optimizer.compute_gradients(deep.loss)
    train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

    grad_summaries = []
    for g, v in grads_and_vars:
        if g is not None:
            grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
            sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
            grad_summaries.append(grad_hist_summary)
            grad_summaries.append(sparsity_summary)
    grad_summaries_merged = tf.summary.merge(grad_summaries)

    # out_dir = os.path.join('/content/drive/My Drive/dataset/chekpoint_music/run', current_time)
    out_dir='run/'+current_time
    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", deep.loss)
    acc_summary = tf.summary.scalar("RMSE", deep.RMSE)
    MAE_summary = tf.summary.scalar("MAE", deep.MAE)
    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, acc_summary,MAE_summary,grad_summaries_merged])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Dev summaries
    dev_summary_op = tf.summary.merge([loss_summary, acc_summary,MAE_summary])
    dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
    dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()     

    # load any vectors from the word2vec         
    sess.run(deep.W1.assign(initW1))   
    sess.run(deep.W2.assign(initW2))

    epoch = 15
    best_mae =5
    best_rmse =5
    train_mae = 0
    train_rmse = 0       
    batch_size=100

    total_batch_train = int(train_length / batch_size) #40
    total_batch_test = int(test_length / batch_size)  #5  
    print("total_batch_num: ",total_batch_train,"\n")
    for epoch in range(epoch):
        # Shuffle the data at each epoch
        shuffle_indices = np.random.permutation(np.arange(train_length))
        shuffled_data = train_data[shuffle_indices]
        datetime_ist = datetime.now(IST) 
        time_str= datetime_ist.strftime('%H:%M:%S')
        print(time_str," epoch_"+str(epoch),"started... ")            
                
        for batch_num in range(total_batch_train):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, train_length)
            data_train = shuffled_data[start_index:end_index]
            uid, iid, y_batch = zip(*data_train) # reuid hamun reid_user
            u_batch = []
            i_batch = []
            for i in range(len(uid)):
                u_batch.append(u_text[uid[i][0]])
                i_batch.append(i_text[iid[i][0]])
            u_batch = np.array(u_batch)
            i_batch = np.array(i_batch)
            t_rmse, t_mae, fm,Train_prediction,y = train_step(u_batch, i_batch, uid, iid, y_batch,batch_num)
           
            current_step = tf.train.global_step(sess, global_step)
            train_rmse += t_rmse
            train_mae += t_mae

            if batch_num%10 == 0 and batch_num > 1:                                                     
                loss_s = 0
                accuracy_s = 0
                mae_s = 0

                for batch_num in range(total_batch_test):
                    start_index = batch_num * batch_size
                    end_index = min((batch_num + 1) * batch_size, test_length)
                    data_test = test_data[start_index:end_index]

                    userid_valid, itemid_valid, y_valid = zip(*data_test)
                    u_valid = []
                    i_valid = []
                    for i in range(len(userid_valid)):
                        u_valid.append(u_text[userid_valid[i][0]])
                        i_valid.append(i_text[itemid_valid[i][0]])
                    u_valid = np.array(u_valid)
                    i_valid = np.array(i_valid)

                    loss, accuracy, mae,prediction,y = dev_step(u_valid, i_valid, userid_valid, itemid_valid, y_valid,writer=dev_summary_writer)
                    loss_s = loss_s + len(u_valid) * loss
                    accuracy_s = accuracy_s + len(u_valid) * np.square(accuracy)
                    mae_s = mae_s + len(u_valid) * mae

                rmse = np.sqrt(accuracy_s/test_length)  #baraye hameye valid ha 
                mae = mae_s / test_length
                if best_rmse > rmse:              #age koochiktar peyda kardi jaygozin kon
                    best_rmse = rmse
                if best_mae > mae:
                    best_mae = mae
        datetime_ist = datetime.now(IST) 
        time_str= datetime_ist.strftime('%H:%M:%S')    
        print (time_str," epoch_"+str(epoch)+" ended.\n" +"Evaluation:")

        print ("train:rmse= {:g}, mae= {:g}".format(train_rmse/total_batch_train , train_mae/total_batch_train))   #bad az hameye batches     
        train_rmse =0
        train_mae = 0
        loss_s = 0
        accuracy_s = 0
        mae_s = 10

        for batch_num in range(total_batch_test):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, test_length)
            data_test = test_data[start_index:end_index]

            userid_valid, itemid_valid, y_valid = zip(*data_test)
            u_valid = []
            i_valid = []
            for i in range(len(userid_valid)):
                u_valid.append(u_text[userid_valid[i][0]])
                i_valid.append(i_text[itemid_valid[i][0]])
            u_valid = np.array(u_valid)
            i_valid = np.array(i_valid)

            loss, accuracy, mae,prediction,y = dev_step(u_valid, i_valid, userid_valid, itemid_valid, y_valid,writer=dev_summary_writer)

            loss_s = loss_s + len(u_valid) * loss
            accuracy_s = accuracy_s + len(u_valid) * np.square(accuracy)
            mae_s = mae_s + len(u_valid) * mae
        print ("valid:rmse= {:g}, mae= {:g}".format(np.sqrt(accuracy_s / test_length), mae_s / test_length))
        
        #                 path = saver.save(sess, checkpoint_prefix, global_step=current_step)
#                 print("Saved model checkpoint to {}\n".format(path))
        rmse = np.sqrt(accuracy_s / test_length)
        mae = mae_s / test_length
        if best_rmse > rmse:
            best_rmse = rmse
        if best_mae > mae:
            best_mae = mae

        if ((epoch+1)%10==0 and epoch>1):
            saved_path = saver.save(sess, '/content/drive/My Drive/dataset/chekpoint_music/new/music', global_step=global_step)
            print('model saved in {}'.format(saved_path))
       
    print ('best rmse:', best_rmse)
    print ('best mae:', best_mae)

#163939 moheme
#104515 moheme

input user shape: (?, 9, 294)
self.embedded_users.shape (?, 9, 294, 100)
self.embedded_item.shape (?, 16, 295, 100)
user jadid shape:  (?, 294, 100)
item jadid shape:  (?, 295, 100)
rnn_outputs_user shape (TensorShape([Dimension(294), Dimension(100)]), TensorShape([Dimension(294), Dimension(100)]))
rnn_outputs_item shape (TensorShape([Dimension(295), Dimension(100)]), TensorShape([Dimension(295), Dimension(100)]))
user_dropOut (?, 294, 100)
item_dropOut (?, 295, 100)
flat output user: (?, 2646, 100)
flat output item: (?, 4720, 100)
attention_score_user (?, 2646, 1)
attention_score_item (?, 4720, 1)
user_feature vector: (?, 100)
item_feature vector: (?, 100)
Xu+qu (?, 32)
Yi+pi (?, 32)
FM  (?, 32)
score:  (?, 1)
self.u_bias (?, 1)
predictions (?, 1)
total_batch_num:  160 

17:56:06  epoch_0 started... 
18:00:27  epoch_0 ended.
Evaluation:
train:rmse= 1.36711, mae= 1.10408
valid:rmse= 1.234, mae= 0.971679
18:00:34  epoch_1 started... 
18:04:53  epoch_1 ended.
Evaluation:
train:rmse= 1.12

In [ ]:
w#resume training
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime 
import pytz 
  
IST = pytz.timezone('Asia/Tehran') 
datetime_ist = datetime.now(IST) 
current_time = datetime_ist.strftime('%H%M%S')
import time


# session_conf = tf.ConfigProto(allow_soft_placement=allow_soft_placement, log_device_placement=log_device_placement)
# session_conf.gpu_options.allow_growth = True
# sess = tf.Session(config=session_conf)
with tf.Session() as sess: 
    saver = tf.train.import_meta_graph('/content/drive/My Drive/dataset/chekpoint_music/new/music-1600.meta')
    saver.restore(sess,tf.train.latest_checkpoint('/content/drive/My Drive/dataset/chekpoint_music/new'))

    
    deep = NARRE( review_num_u=review_num_u, review_num_i=review_num_i,  review_len_u=review_len_u,  review_len_i=review_len_i,
        user_num=user_num, item_num=item_num, num_classes=1, user_vocab_size=len(vocabulary_user), item_vocab_size=len(vocabulary_item),
        embedding_size=embedding_dim,  embedding_id=32,l2_reg_lambda=l2_reg_lambda, attention_size=32,  n_latent=32)
    
    global_step = tf.Variable(1600, name="global_step", trainable=False)
    # optimizer = tf.train.AdagradOptimizer(learning_rate=0.01, initial_accumulator_value=1e-8).minimize(deep.loss)
    optimizer =tf.train.AdamOptimizer(0.006) # tf.train.AdamOptimizer(0.002, beta1=0.9, beta2=0.999, epsilon=1e-8).minimize(deep.loss)
    grads_and_vars = optimizer.compute_gradients(deep.loss)
    train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
    saver.load

    grad_summaries = []
    for g, v in grads_and_vars:
        if g is not None:
            grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
            sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
            grad_summaries.append(grad_hist_summary)
            grad_summaries.append(sparsity_summary)
    grad_summaries_merged = tf.summary.merge(grad_summaries)

    # out_dir = os.path.join('/content/drive/My Drive/dataset/chekpoint_music/run', current_time)
    out_dir='run/'+current_time
    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", deep.loss)
    acc_summary = tf.summary.scalar("RMSE", deep.RMSE)
    MAE_summary = tf.summary.scalar("MAE", deep.MAE)
    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, acc_summary,MAE_summary,grad_summaries_merged])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Dev summaries
    dev_summary_op = tf.summary.merge([loss_summary, acc_summary,MAE_summary])
    dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
    dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
    # sess.run(tf.global_variables_initializer())
    # sess.run(train_op)
    saver = tf.train.Saver()     

    # load any vectors from the word2vec         
    sess.run(deep.W1.assign(initW1))   
    sess.run(deep.W2.assign(initW2))
    epoch = 10
    best_mae =5
    best_rmse =5
    train_mae = 0
    train_rmse = 0       
    batch_size=100

    total_batch_train = int(train_length / batch_size) #40
    total_batch_test = int(test_length / batch_size)  #5  
    print("total_batch_num: ",total_batch_train,"\n")
    for epoch in range(epoch):
        # Shuffle the data at each epoch
        shuffle_indices = np.random.permutation(np.arange(train_length))
        shuffled_data = train_data[shuffle_indices]
        datetime_ist = datetime.now(IST) 
        time_str= datetime_ist.strftime('%H:%M:%S')
        print(time_str," epoch_"+str(epoch),"started... ")            
                
        for batch_num in range(total_batch_train):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, train_length)
            data_train = shuffled_data[start_index:end_index]
            uid, iid, y_batch = zip(*data_train) # reuid hamun reid_user
            u_batch = []
            i_batch = []
            for i in range(len(uid)):
                u_batch.append(u_text[uid[i][0]])
                i_batch.append(i_text[iid[i][0]])
            u_batch = np.array(u_batch)
            i_batch = np.array(i_batch)
            t_rmse, t_mae, fm,Train_prediction,y = train_step(u_batch, i_batch, uid, iid, y_batch,batch_num)
            print('after train step')
            current_step = tf.train.global_step(sess, global_step)
            train_rmse += t_rmse
            train_mae += t_mae




input user shape: (?, 9, 294)
self.embedded_users.shape (?, 9, 294, 100)
self.embedded_item.shape (?, 16, 295, 100)
user jadid shape:  (?, 294, 100)
item jadid shape:  (?, 295, 100)
rnn_outputs_user shape (TensorShape([Dimension(294), Dimension(100)]), TensorShape([Dimension(294), Dimension(100)]))
rnn_outputs_item shape (TensorShape([Dimension(295), Dimension(100)]), TensorShape([Dimension(295), Dimension(100)]))
user_dropOut (?, 294, 100)
item_dropOut (?, 295, 100)
flat output user: (?, 2646, 100)
flat output item: (?, 4720, 100)
attention_score_user (?, 2646, 1)
attention_score_item (?, 4720, 1)
user_feature vector: (?, 100)
item_feature vector: (?, 100)
Xu+qu (?, 32)
Yi+pi (?, 32)
FM  (?, 32)
score:  (?, 1)
self.u_bias (?, 1)
predictions (?, 1)
total_batch_num:  160 

19:06:02  epoch_0 started... 
after feed dict


FailedPreconditionError: ignored

In [ ]:
saved_path = saver.save(sess, '/content/drive/My Drive/dataset/chekpoint_music/new/musicLast')
print('model saved in {}'.format(saved_path))

model saved in /content/drive/My Drive/dataset/chekpoint_music/new/musicLast


In [ ]:
from sklearn.metrics import precision_score,recall_score,precision_recall_curve

precision= precision_score(y,prediction.astype(int),average='weighted')
recall=recall_score(y,prediction.astype(int),average='weighted')

In [ ]:
!kill 366

In [ ]:
%reload_ext tensorboard

%tensorboard --logdir='/content/drive/My Drive/dataset/chekpoint_music/run/104515/summaries/train'

In [ ]:
vars_list = tf.train.list_variables('/content/drive/My Drive/dataset/chekpoint_music/new') #just variables 
vars_list

In [ ]:
#test model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()

pkl_file = open('/content/drive/My Drive/dataset/music/title/music.train', 'rb')
train_data = pickle.load(pkl_file)


with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('/content/drive/My Drive/dataset/chekpoint_music/new/music-6880.meta')
    saver.restore(sess,tf.train.latest_checkpoint('/content/drive/My Drive/dataset/chekpoint_music/new'))

    graph = tf.get_default_graph()
    input_u = graph.get_tensor_by_name('input_u:0')
    input_i = graph.get_tensor_by_name('input_i:0')
    input_y = graph.get_tensor_by_name('input_y:0')
    input_uid = graph.get_tensor_by_name('input_uid:0')
    input_iid = graph.get_tensor_by_name('input_iid:0')
    prediction = graph.get_tensor_by_name("prediction/predictions:0")
    score= graph.get_tensor_by_name("attention_user/attention_score_user:0")
    
    data_train = train_data[:100]
    uid, iid, y_batch = zip(*data_train) # reuid hamun reid_user
    u_batch = []
    i_batch = []
    for i in range(len(uid)):
        u_batch.append(u_text[uid[i][0]])
        i_batch.append(i_text[iid[i][0]])
    u_batch = np.array(u_batch)
    i_batch = np.array(i_batch)

    feed_dict = {                         #placeholder haro meghdardehi sabet mikone
        input_u: u_batch,
        input_i: i_batch,
        input_uid: uid,
        input_iid: iid,
        input_y: y_batch,
    }
    prediction = sess.run(prediction,feed_dict)
    attention_score=sess.run(score,feed_dict)
    

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/dataset/chekpoint_music/new/music-6880


In [ ]:
print(uid[0][0])
print(iid[0][0])
print(y_batch[0][0])

In [ ]:
df=pd.read_csv("/content/drive/My Drive/dataset/music/title/music_processed.csv")
# df.loc[df['user_id']==2426]
df["reviews"][17839]

In [ ]:
# np.where(attention_score[0] == np.amax(attention_score[0]))
np.amax(attention_score[0][:10])

0.000978405

In [ ]:
print(attention_score[0][:10].round(decimals=10))

In [ ]:
#number of wrong answers
np.sum(np.array(prediction).round() != y_batch)


25

In [ ]:
print(*prediction[:5])
print(*y_batch[:5])

[5.3418446] [5.319431] [3.9605584] [4.451573] [5.1865077]
[5.] [5.] [5.] [4.] [5.]


In [ ]:
#recommendation for a user
uid, iid, y_batch = zip(*train_data[:100])
for i in range(100):
  if(uid[i][0]==2426):
    if(prediction[i]>4.0):
      print(iid[i][0] )


1563


In [ ]:
import pandas as pd
title= pd.read_csv("/content/drive/My Drive/dataset/music/title/music_processed.csv")
title.head()
for a in title.values:
  if (a[1]==1563):
    print(a[4])

In [ ]:
df = pd.merge(data2, dfTitle, how='right', on=['item_id'])
df=df[['user_id','item_id','reviews','Ratings','title']]
df.shape

(58490, 5)